<a href="https://colab.research.google.com/github/ravikumar1478/masked_language_modeling_for_Telugu/blob/main/SUCCESS_NLP_MLM_TELUGU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from datasets import load_dataset
datasets = load_dataset('telugu_books', data_dir = '/content/gdrive/MyDrive/')

In [ ]:
datasets["train"][10]

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

In [ ]:
model_checkpoint = "xlm-roberta-base"

In [ ]:
from transformers import AutoTokenizer

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/7 [00:00<?, ?ba/s]

#1:   0%|          | 0/7 [00:00<?, ?ba/s]

#2:   0%|          | 0/7 [00:00<?, ?ba/s]

#3:   0%|          | 0/7 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1192 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (874 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (937 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (782 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=500,
    num_proc=4,
)

#0:   0%|          | 0/13 [00:00<?, ?ba/s]

#1:   0%|          | 0/13 [00:00<?, ?ba/s]

#2:   0%|          | 0/13 [00:00<?, ?ba/s]

#3:   0%|          | 0/13 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=lm_datasets["train"].select(range(10000)),
eval_dataset=lm_datasets["train"].select(range(1500)),
data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 278295186
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.419200,2.155717
2,2.285900,2.063224
3,2.231100,2.008281


Saving model checkpoint to xlm-roberta-base-finetuned-wikitext2/checkpoint-500
Configuration saved in xlm-roberta-base-finetuned-wikitext2/checkpoint-500/config.json
Model weights saved in xlm-roberta-base-finetuned-wikitext2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to xlm-roberta-base-finetuned-wikitext2/checkpoint-1000
Configuration saved in xlm-roberta-base-finetuned-wikitext2/checkpoint-1000/config.json
Model weights saved in xlm-roberta-base-finetuned-wikitext2/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8
Saving model checkpoint to xlm-roberta-base-finetuned-wikitext2/checkpoint-1500
Configuration saved in xlm-roberta-base-finetuned-wikitext2/checkpoint-1500/config.json
Model weights saved in xlm-roberta-base-finetuned-wikitext2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to xlm-roberta-base-finetuned-wikitext2/checkpoint-2000
Configuration saved in xlm-roberta-base-finetuned-wikitext2/checkpo

TrainOutput(global_step=3750, training_loss=2.336316064453125, metrics={'train_runtime': 2542.1276, 'train_samples_per_second': 11.801, 'train_steps_per_second': 1.475, 'total_flos': 1979092915200000.0, 'train_loss': 2.336316064453125, 'epoch': 3.0})

In [ ]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained("ravikumar1478/xlm-roberta-base-finetuned-wikitext2")

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ravikumar1478--xlm-roberta-base-finetuned-wikitext2/snapshots/31550c0c8091b582e45b26019f4e33d8f3dbda93/config.json
You are using a model of type xlm-roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Model config BertConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_versi

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--ravikumar1478--xlm-roberta-base-finetuned-wikitext2/snapshots/31550c0c8091b582e45b26019f4e33d8f3dbda93/pytorch_model.bin
Some weights of the model checkpoint at ravikumar1478/xlm-roberta-base-finetuned-wikitext2 were not used when initializing BertForMaskedLM: ['roberta.encoder.layer.4.attention.self.value.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.11.attention.output.dense.bias', 'roberta.encoder.layer.8.attention.self.value.weight', 'roberta.encoder.layer.5.intermediate.dense.weight', 'roberta.encoder.layer.4.attention.output.LayerNorm.bias', 'roberta.encoder.layer.10.output.dense.bias', 'roberta.encoder.layer.9.output.dense.weight', 'roberta.encoder.layer.3.attention.output.dense.bias', 'roberta.encoder.layer.9.intermediate.dense.weight', 'lm_head.layer_norm.weight', 'roberta.encoder.layer.3.attention.output.dense.weight', 'roberta.encoder.layer.6

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8


Perplexity: 7.68


In [ ]:
trainer.push_to_hub()

Saving model checkpoint to xlm-roberta-base-finetuned-wikitext2
Configuration saved in xlm-roberta-base-finetuned-wikitext2/config.json
Model weights saved in xlm-roberta-base-finetuned-wikitext2/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.04G [00:00<?, ?B/s]

Upload file runs/Nov29_12-37-30_571c98ab0f6f/events.out.tfevents.1669728133.571c98ab0f6f.79.2: 100%|##########…

Upload file runs/Nov29_12-37-30_571c98ab0f6f/events.out.tfevents.1669725528.571c98ab0f6f.79.0:  56%|#####6    …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/ravikumar1478/xlm-roberta-base-finetuned-wikitext2
   0cab677..6f2b911  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/ravikumar1478/xlm-roberta-base-finetuned-wikitext2
   0cab677..6f2b911  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
To https://huggingface.co/ravikumar1478/xlm-roberta-base-finetuned-wikitext2
   6f2b911..31550c0  main -> main

   6f2b911..31550c0  main -> main



'https://huggingface.co/ravikumar1478/xlm-roberta-base-finetuned-wikitext2/commit/6f2b911ac1ceb79d5dd9cc27980e70ee69f8c744'

In [ ]:
tokenizer.push_to_hub("ravikumar1478/xlm-roberta-base-finetuned-wikitext2")

tokenizer config file saved in xlm-roberta-base-finetuned-wikitext2/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-wikitext2/special_tokens_map.json
Uploading the following files to ravikumar1478/xlm-roberta-base-finetuned-wikitext2: tokenizer_config.json,tokenizer.json,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/ravikumar1478/xlm-roberta-base-finetuned-wikitext2/commit/447708c16519f14c3aa83b8a7cf5591a81f0cf0a', commit_message='Upload tokenizer', commit_description='', oid='447708c16519f14c3aa83b8a7cf5591a81f0cf0a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer._tokenizer.save("tokenizer.json")